# Data Cleaning and Merging Notebook

Notes on data cleaning / merging. The objective of the work in this notebook is to merge the various metrics we want to use into one dataset which we can then use for run analysis on. We will not at this stage be removing null values as where one metric might be missing values another will not and depending on what metrics we're looking to run the analysis we might not care about data missing in another metric. Removing null values will happen at the analysis point depending on what analysis / relationships is being explored. Hence most mergers are done as 'outer' merges.

Only data which is removed at this stage is data for years earlier than 2010, as that is when our main Internet Penetration dataset starts and countries which are not included in the Internet Penetration dataset.

Data is sourced through CSVs and APIs, some challenges included: different spellings of countries so needing to join several datasets with ISO codes to match properly.

## Table of Contents
1. [Internet Penetration Data Cleaning - csv](#internetcleaning)
2. [GNI Data Cleaning and Merging - csv](#gnicleaning)
3. [GINI Index Data Cleaning and Merging - csv](#ginicleaning)
4. [GDP Data Cleaning and Merging - csv](#gdpcleaning)
5. [Happiness Index Cleaning and Merging - csv](#happinesindexcleaning)
6. [Literacy Rates Cleaning and Merging - api](#literacyratescleaning)
7. [Suicide Rates Cleaning and Merging - api](#suicideratescleaning)
8. [CO2 Emissions Cleaning and Merging - api](#co2cleaning)
9. [Internet Price Data Cleaning and Merging](#ipcleaning)
10. [Voter Turnout Cleaning and Merging](#VoterTurnout)
11. [Final Data Export](#finalexport)


## Internet Penetration Data Cleaning <a name="internetcleaning"></a>
Pulling in and setting up the data from the ITU dataset on Internet Peneration rates by country by year

In [1]:
import requests
import pandas as pd

internet_data = pd.read_excel("data/ddd_dataset.xlsx")
internet_data.head()

,Country,Region,ISO,Indicator name,Year,Value,ITU estimate?
0,Afghanistan,Asia & Pacific,AFG,Active mobile-broadband subscriptions per 100 ...,2010,0.000000,NaN
1,Afghanistan,Asia & Pacific,AFG,Fixed broadband subscriptions per 100 inhabitants,2010,0.005321,NaN
2,Afghanistan,Asia & Pacific,AFG,Fixed-telephone subscriptions per 100 inhabitants,2010,0.058958,NaN
3,Afghanistan,Asia & Pacific,AFG,"Individuals using the Internet, total (%)",2010,4.000000,yes
4,Afghanistan,Asia & Pacific,AFG,International bandwidth per Internet user (kbi...,2010,1773.699247,NaN


In [2]:
internet_data.shape

(41045, 7)

In [3]:
internet_data_pivot = internet_data.pivot_table(index=['Country', 'Region', 'ISO', 'Year'], columns='Indicator name', values='Value').reset_index()
internet_data_pivot.columns


Index(['Country', 'Region', 'ISO', 'Year',
       'Active mobile-broadband subscriptions per 100 inhabitants',
       'Female Internet users as a % of total female population',
       'Female mobile phone ownership as a % of total female population',
       'Fixed broadband basket as a % of GNI p.c.',
       'Fixed broadband subscriptions per 100 inhabitants',
       'Fixed broadband subscriptions: 2 to 10 Mbit/s',
       'Fixed broadband subscriptions: 256kbit/s - <2Mbit/s',
       'Fixed broadband subscriptions: >10 Mbit/s',
       'Fixed-telephone subscriptions per 100 inhabitants',
       'Households with Internet access at home (%)',
       'Households with Internet access at home, rural (%)',
       'Households with Internet access at home, urban (%)',
       'Households with a computer at home (%)',
       'Individuals owning a mobile phone (%)',
       'Individuals using the Internet, total (%)',
       'International bandwidth per Internet user (kbit/s)',
       'Internet user

In [4]:
columns_keep = [
    'Country', 'Region', 'ISO', 'Year',
    'Individuals using the Internet, total (%)'
]

internet_df = internet_data_pivot.loc[:, columns_keep]
internet_df

Indicator name,Country,Region,ISO,Year,"Individuals using the Internet, total (%)"
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000
...,...,...,...,...,...
2531,Zimbabwe,Africa,ZWE,2018,25.000000
2532,Zimbabwe,Africa,ZWE,2019,26.588274
2533,Zimbabwe,Africa,ZWE,2020,29.298565
2534,Zimbabwe,Africa,ZWE,2021,34.813971


In [5]:
internet_df.describe()

Indicator name,Year,"Individuals using the Internet, total (%)"
count,2536.000000,2303.000000
mean,2016.003155,50.239339
std,3.743869,30.182481
min,2010.000000,0.000000
25%,2013.000000,22.126371
50%,2016.000000,52.570000
75%,2019.000000,77.793618
max,2022.000000,100.000000


In [6]:
internet_df.shape

(2536, 5)

Name of final dataframe at this stage is internet_df

## GNI Data Cleaning and Merging <a name="gnicleaning"></a>
Pulling in and setting up the data from the world bank dataset on GNI index by country by year.


In [7]:
gni_initial_dataset = pd.read_csv("data/API_NY.GNP.PCAP.CD_DS2_en_csv_v2_5995153.csv", skiprows=4)
gni_initial_dataset.describe()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
count,0.0,0.0,91.000000,95.000000,100.000000,100.000000,106.000000,116.000000,121.000000,123.000000,...,246.000000,247.000000,246.000000,247.000000,247.000000,247.000000,244.000000,242.000000,230.000000,0.0
mean,NaN,NaN,531.029978,548.675836,577.470411,625.984096,655.325566,678.725735,732.203761,777.301303,...,14614.966827,13906.813772,13499.390230,13604.672333,14521.260807,15269.493880,14246.927678,15243.050960,15634.511279,NaN
std,NaN,NaN,668.348745,696.089995,739.005508,803.617720,855.350719,889.517219,963.207138,1035.314350,...,19999.177897,18883.747206,18275.606286,18435.882072,19611.637771,20239.705223,19210.726095,20485.527531,21309.703328,NaN
min,NaN,NaN,40.000000,30.000000,30.000000,20.000000,20.000000,10.000000,10.000000,20.000000,...,250.000000,250.000000,250.000000,250.000000,240.000000,230.000000,220.000000,220.000000,240.000000,NaN
25%,NaN,NaN,116.540047,120.000000,130.000000,138.121516,150.000000,155.878350,162.886992,165.000000,...,2005.509921,2009.744307,2020.861033,1990.000000,1980.000000,2095.000000,2090.000000,2177.500000,2234.391723,NaN
50%,NaN,NaN,230.000000,230.000000,225.000000,250.000000,270.000000,280.000000,300.000000,300.000000,...,6272.843434,5960.000000,5695.000000,5770.000000,6160.000000,6505.791615,6010.000000,6344.792244,6770.000000,NaN
75%,NaN,NaN,600.000000,630.000000,670.000000,707.500000,725.000000,720.000000,750.000000,805.000000,...,16065.000000,15960.000000,16342.500000,16460.000000,17320.000000,18000.000000,16795.000000,17995.000000,18845.000000,NaN
max,NaN,NaN,3280.000000,3410.000000,3610.000000,3880.000000,4190.000000,4370.000000,4740.000000,5070.000000,...,105070.000000,101120.000000,103620.000000,112940.000000,117000.000000,117280.000000,111800.000000,120020.000000,125240.000000,NaN


In [8]:
gni_initial_dataset.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,25790.000000,26180.000000,26650.000000,27720.000000,29310.000000,30330.000000,24840.000000,29460.000000,NaN,NaN
1,Africa Eastern and Southern,AFE,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1696.462899,1579.295342,1460.609129,1443.996026,1456.396534,1494.589953,1388.531225,1461.388019,1542.260985,NaN
2,Afghanistan,AFG,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,650.000000,610.000000,570.000000,540.000000,520.000000,530.000000,500.000000,390.000000,NaN,NaN
3,Africa Western and Central,AFW,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,NaN,NaN,113.080093,121.446289,127.515314,132.486065,...,2101.336153,2009.488614,1776.140878,1590.799458,1606.960024,1681.824079,1664.742718,1727.603209,1779.431390,NaN
4,Angola,AGO,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,4800.000000,3880.000000,2450.000000,1980.000000,1870.000000,2040.000000,1690.000000,1650.000000,1900.000000,NaN


In [9]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
gni_initial_dataset.isnull().sum()

Country Name        0
Country Code        0
Indicator Name      0
Indicator Code      0
1960              266
1961              266
1962              175
1963              171
1964              166
1965              166
1966              160
1967              150
1968              145
1969              143
1970              144
1971              144
1972              133
1973              132
1974              132
1975              133
1976              132
1977              131
1978              128
1979              123
1980              118
1981              117
1982              104
1983               99
1984               96
1985               94
1986               91
1987               91
1988               89
1989               86
1990               83
1991               82
1992               70
1993               64
1994               61
1995               52
1996               50
1997               40
1998               40
1999               40
2000               37
2001      

Given we only have internet penetration data from 2010 and the effects of internet would take some time be realised, we'll definitely get rid of all the data from before 2010 as thats where most of the data is missing as well as the column called 'unnamed: 67' as it's all blanks.

In [10]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
gni_unpivot_dataset = pd.melt(gni_initial_dataset, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
                       var_name='Year', value_name='Value')

gni_unpivot_dataset

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
0,Aruba,ABW,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,1960,NaN
1,Africa Eastern and Southern,AFE,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,1960,NaN
2,Afghanistan,AFG,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,1960,NaN
3,Africa Western and Central,AFW,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,1960,NaN
4,Angola,AGO,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,1960,NaN
...,...,...,...,...,...,...
17019,Kosovo,XKX,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,Unnamed: 67,NaN
17020,"Yemen, Rep.",YEM,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,Unnamed: 67,NaN
17021,South Africa,ZAF,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,Unnamed: 67,NaN
17022,Zambia,ZMB,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,Unnamed: 67,NaN


In [11]:

# gni_relevant_years = gni_unpivot_dataset[(gni_unpivot_dataset['Year'] >= 2010) & (gni_unpivot_dataset['Year'] != 'Unnamed: 67')]
# gni_relevant_years

# Not able to use this approach as the year is stored as a string so will get rid of the Unnamed:67 rows first and then convert that column into integer values
gni_years_only = gni_unpivot_dataset[(gni_unpivot_dataset['Year'] != 'Unnamed: 67')]
gni_years_only

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
0,Aruba,ABW,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,1960,NaN
1,Africa Eastern and Southern,AFE,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,1960,NaN
2,Afghanistan,AFG,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,1960,NaN
3,Africa Western and Central,AFW,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,1960,NaN
4,Angola,AGO,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,1960,NaN
...,...,...,...,...,...,...
16753,Kosovo,XKX,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,2022,5590.0
16754,"Yemen, Rep.",YEM,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,2022,NaN
16755,South Africa,ZAF,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,2022,6780.0
16756,Zambia,ZMB,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,2022,1170.0


Now changing the year column to integers so we can cut out the years but a simple less than argument

In [12]:
gni_years_only.loc[:, 'Year'] = gni_years_only['Year'].astype(int)
gni_years_only.dtypes

Country Name       object
Country Code       object
Indicator Name     object
Indicator Code     object
Year               object
Value             float64
dtype: object

Have two options here for changing the data type of integer as sometimes one of them don't work.

In [13]:
gni_years_only['Year'] = pd.to_numeric(gni_years_only['Year'], errors='coerce')
gni_years_only.dtypes

/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/3982770618.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gni_years_only['Year'] = pd.to_numeric(gni_years_only['Year'], errors='coerce')


Country Name       object
Country Code       object
Indicator Name     object
Indicator Code     object
Year                int64
Value             float64
dtype: object

Now that the year column is converted we can create a new dataset filtering out the years we don't need.

In [14]:
gni_relevant_years = gni_years_only[(gni_years_only['Year'] >= 2010)]
gni_relevant_years

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
13300,Aruba,ABW,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,2010,23390.000000
13301,Africa Eastern and Southern,AFE,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,2010,1438.818933
13302,Afghanistan,AFG,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,2010,520.000000
13303,Africa Western and Central,AFW,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,2010,1559.231753
13304,Angola,AGO,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,2010,3210.000000
...,...,...,...,...,...,...
16753,Kosovo,XKX,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,2022,5590.000000
16754,"Yemen, Rep.",YEM,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,2022,NaN
16755,South Africa,ZAF,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,2022,6780.000000
16756,Zambia,ZMB,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,2022,1170.000000


In [15]:
merged_df = pd.merge(internet_df, gni_relevant_years[['Country Code', 'Year', 'Value']], how='outer', left_on=['ISO', 'Year'], right_on=['Country Code', 'Year'])
merged_df = merged_df.rename(columns={'Value': 'GNI per capita'})
merged_df


,Country,Region,ISO,Year,"Individuals using the Internet, total (%)",Country Code,GNI per capita
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,AFG,520.000000
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,AFG,550.000000
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,AFG,650.000000
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,AFG,680.000000
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,AFG,650.000000
...,...,...,...,...,...,...,...
3469,NaN,NaN,NaN,2022,NaN,UMC,10529.880203
3470,NaN,NaN,NaN,2022,NaN,VGB,NaN
3471,NaN,NaN,NaN,2022,NaN,VIR,NaN
3472,NaN,NaN,NaN,2022,NaN,WLD,12803.872147


The GNI Index had some regional groups, socioeconomic groups and countries with the UN doesn't recognize and weren't included in the internet penetration dataset so I want to remove all the rows where the Country / Region / ISO is blank, they seem to all be the same rows, 938 of them.


In [16]:
empty_country_df = merged_df[merged_df['Country'].isnull()]
empty_country_df

,Country,Region,ISO,Year,"Individuals using the Internet, total (%)",Country Code,GNI per capita
2536,NaN,NaN,NaN,2010,NaN,ABW,23390.000000
2537,NaN,NaN,NaN,2010,NaN,AFE,1438.818933
2538,NaN,NaN,NaN,2010,NaN,AFW,1559.231753
2539,NaN,NaN,NaN,2010,NaN,ARB,6168.562166
2540,NaN,NaN,NaN,2010,NaN,ASM,NaN
...,...,...,...,...,...,...,...
3469,NaN,NaN,NaN,2022,NaN,UMC,10529.880203
3470,NaN,NaN,NaN,2022,NaN,VGB,NaN
3471,NaN,NaN,NaN,2022,NaN,VIR,NaN
3472,NaN,NaN,NaN,2022,NaN,WLD,12803.872147


Finding all codes which don't have corresponding code the internet data and the creating a dictionary to make sure we're happy to drop all those areas and that they do in fact correspond to regions / sub regions / aggregates etc.

In [17]:
codes_to_drop = []
country_codes = merged_df['Country Code'].tolist()
iso_codes = merged_df['ISO'].tolist()

for code in country_codes:
    if code not in iso_codes:
        codes_to_drop.append(code)

country_code_country_dict = dict(zip(gni_relevant_years['Country Code'], gni_relevant_years['Country Name']))

filtered_dict = {key: value for key, value in country_code_country_dict.items() if key in codes_to_drop}
filtered_dict

{'ABW': 'Aruba',
 'AFE': 'Africa Eastern and Southern',
 'AFW': 'Africa Western and Central',
 'ARB': 'Arab World',
 'ASM': 'American Samoa',
 'BMU': 'Bermuda',
 'CEB': 'Central Europe and the Baltics',
 'CHI': 'Channel Islands',
 'CSS': 'Caribbean small states',
 'CUW': 'Curacao',
 'CYM': 'Cayman Islands',
 'EAP': 'East Asia & Pacific (excluding high income)',
 'EAR': 'Early-demographic dividend',
 'EAS': 'East Asia & Pacific',
 'ECA': 'Europe & Central Asia (excluding high income)',
 'ECS': 'Europe & Central Asia',
 'EMU': 'Euro area',
 'EUU': 'European Union',
 'FCS': 'Fragile and conflict affected situations',
 'FRO': 'Faroe Islands',
 'GIB': 'Gibraltar',
 'GRL': 'Greenland',
 'GUM': 'Guam',
 'HIC': 'High income',
 'HPC': 'Heavily indebted poor countries (HIPC)',
 'IBD': 'IBRD only',
 'IBT': 'IDA & IBRD total',
 'IDA': 'IDA total',
 'IDB': 'IDA blend',
 'IDX': 'IDA only',
 'IMN': 'Isle of Man',
 'INX': 'Not classified',
 'LAC': 'Latin America & Caribbean (excluding high income)',
 

Having scanned this filtered list dictionary, happy to remove all rows corresponding to these country codes in the merged dataset.

In [18]:
internet_gni_df = merged_df[~merged_df['Country Code'].isin(codes_to_drop)]
internet_gni_df

,Country,Region,ISO,Year,"Individuals using the Internet, total (%)",Country Code,GNI per capita
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,AFG,520.0
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,AFG,550.0
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,AFG,650.0
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,AFG,680.0
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,AFG,650.0
...,...,...,...,...,...,...,...
2533,Zimbabwe,Africa,ZWE,2020,29.298565,ZWE,1460.0
2534,Zimbabwe,Africa,ZWE,2021,34.813971,ZWE,1530.0
2535,Zimbabwe,Africa,ZWE,2022,NaN,ZWE,1500.0
3129,NaN,NaN,NaN,2018,NaN,ERI,NaN


Now need to fill in Country, Region, ISO columns where we have the GNI metric, or at a least a row came in from that dataset for that year but that country / year combination wasn't in the dataset originally.

In [19]:
country_code_info_dict = dict(zip(internet_gni_df['ISO'], zip(internet_gni_df['Country'], internet_gni_df['Region'])))
country_code_info_dict

{'AFG': ('Afghanistan', 'Asia & Pacific'),
 'ALB': ('Albania', 'Europe'),
 'DZA': ('Algeria', 'Arab States'),
 'AND': ('Andorra', 'Europe'),
 'AGO': ('Angola', 'Africa'),
 'ATG': ('Antigua and Barbuda', 'The Americas'),
 'ARG': ('Argentina', 'The Americas'),
 'ARM': ('Armenia', 'CIS'),
 'AUS': ('Australia', 'Asia & Pacific'),
 'AUT': ('Austria', 'Europe'),
 'AZE': ('Azerbaijan', 'CIS'),
 'BHS': ('Bahamas', 'The Americas'),
 'BHR': ('Bahrain', 'Arab States'),
 'BGD': ('Bangladesh', 'Asia & Pacific'),
 'BRB': ('Barbados', 'The Americas'),
 'BLR': ('Belarus', 'CIS'),
 'BEL': ('Belgium', 'Europe'),
 'BLZ': ('Belize', 'The Americas'),
 'BEN': ('Benin', 'Africa'),
 'BTN': ('Bhutan', 'Asia & Pacific'),
 'BOL': ('Bolivia (Plurinational State of)', 'The Americas'),
 'BIH': ('Bosnia and Herzegovina', 'Europe'),
 'BWA': ('Botswana', 'Africa'),
 'BRA': ('Brazil', 'The Americas'),
 'BRN': ('Brunei Darussalam', 'Asia & Pacific'),
 'BGR': ('Bulgaria', 'Europe'),
 'BFA': ('Burkina Faso', 'Africa'),
 '

In [20]:
for index, row in internet_gni_df.iterrows():
    iso_code = row["Country Code"]
    if iso_code in country_code_info_dict:
        country_name, region = country_code_info_dict[iso_code]
        if pd.isnull(row["Country"]):
            internet_gni_df.at[index, "Country"] = country_name
        if pd.isnull(row["Region"]):
            internet_gni_df.at[index, "Region"] = region
        if pd.isnull(row["ISO"]):
            internet_gni_df.at[index, "ISO"] = iso_code

internet_gni_df

,Country,Region,ISO,Year,"Individuals using the Internet, total (%)",Country Code,GNI per capita
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,AFG,520.0
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,AFG,550.0
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,AFG,650.0
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,AFG,680.0
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,AFG,650.0
...,...,...,...,...,...,...,...
2533,Zimbabwe,Africa,ZWE,2020,29.298565,ZWE,1460.0
2534,Zimbabwe,Africa,ZWE,2021,34.813971,ZWE,1530.0
2535,Zimbabwe,Africa,ZWE,2022,NaN,ZWE,1500.0
3129,Eritrea,Africa,ERI,2018,NaN,ERI,NaN


checking that it implemented properly

In [21]:
pd.set_option("display.max_rows", None)
internet_gni_df[internet_gni_df["Country Code"]=="ERI"]

,Country,Region,ISO,Year,"Individuals using the Internet, total (%)",Country Code,GNI per capita
714,Eritrea,Africa,ERI,2010,0.610000,ERI,520.0
715,Eritrea,Africa,ERI,2011,0.700000,ERI,610.0
716,Eritrea,Africa,ERI,2012,0.800000,ERI,NaN
717,Eritrea,Africa,ERI,2013,0.900000,ERI,NaN
718,Eritrea,Africa,ERI,2014,0.990000,ERI,NaN
719,Eritrea,Africa,ERI,2015,1.083733,ERI,NaN
720,Eritrea,Africa,ERI,2016,1.177119,ERI,NaN
721,Eritrea,Africa,ERI,2017,1.308907,ERI,NaN
722,Eritrea,Africa,ERI,2019,8.832308,ERI,NaN
723,Eritrea,Africa,ERI,2020,14.179987,ERI,NaN


In [22]:
internet_gni_df.shape

(2538, 7)

Final dataframe after this section is called: internet_gni_df

## GINI Index Data Cleaning <a name="ginicleaning"></a>
Pulling in and setting up the data from the world bank dataset on GINI coefficient by country by year.

In [23]:
pd.set_option('display.max_rows', 10)
gini_initial_data = pd.read_csv("data/API_SI.POV.GINI_DS2_en_csv_v2_5994673.csv", skiprows=4)
gini_initial_data

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,AFG,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Africa Western and Central,AFW,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,AGO,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.3,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,Kosovo,XKX,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.0,NaN,31.2,30.3,NaN,NaN,31.8,33.3,27.8,29.0,26.3,27.3,26.5,26.7,29.0,NaN,NaN,NaN,NaN,NaN,NaN
262,"Yemen, Rep.",YEM,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,34.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,South Africa,ZAF,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.3,NaN,NaN,NaN,NaN,NaN,NaN,57.8,NaN,NaN,NaN,NaN,64.8,NaN,NaN,63.0,NaN,63.4,NaN,NaN,NaN,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
264,Zambia,ZMB,Gini index,SI.POV.GINI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.5,NaN,52.6,NaN,NaN,48.3,NaN,49.1,NaN,NaN,NaN,42.1,NaN,54.3,NaN,54.6,NaN,NaN,NaN,52.0,NaN,NaN,NaN,NaN,55.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
gini_initial_data.isnull().sum()

Country Name        0
Country Code        0
Indicator Name      0
Indicator Code      0
1960              266
1961              266
1962              266
1963              265
1964              265
1965              265
1966              265
1967              265
1968              264
1969              264
1970              263
1971              263
1972              264
1973              263
1974              264
1975              261
1976              264
1977              262
1978              264
1979              259
1980              261
1981              257
1982              261
1983              260
1984              258
1985              252
1986              250
1987              244
1988              253
1989              250
1990              244
1991              247
1992              232
1993              240
1994              240
1995              228
1996              230
1997              236
1998              225
1999              229
2000              217
2001      

Seems to be a lot of data missing in this dataset, might not be able to use it quite as desired but will still reconstruct it and add it for the relevant years (i.e. 2010 onwards).

In [25]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

gini_unpivot_dataset = pd.melt(gini_initial_data, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
                       var_name='Year', value_name='Value')

gini_unpivot_dataset

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
0,Aruba,ABW,Gini index,SI.POV.GINI,1960,NaN
1,Africa Eastern and Southern,AFE,Gini index,SI.POV.GINI,1960,NaN
2,Afghanistan,AFG,Gini index,SI.POV.GINI,1960,NaN
3,Africa Western and Central,AFW,Gini index,SI.POV.GINI,1960,NaN
4,Angola,AGO,Gini index,SI.POV.GINI,1960,NaN
...,...,...,...,...,...,...
17019,Kosovo,XKX,Gini index,SI.POV.GINI,Unnamed: 67,NaN
17020,"Yemen, Rep.",YEM,Gini index,SI.POV.GINI,Unnamed: 67,NaN
17021,South Africa,ZAF,Gini index,SI.POV.GINI,Unnamed: 67,NaN
17022,Zambia,ZMB,Gini index,SI.POV.GINI,Unnamed: 67,NaN


In [26]:
gini_years_only = gini_unpivot_dataset[(gini_unpivot_dataset['Year'] != 'Unnamed: 67')]
gini_years_only

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
0,Aruba,ABW,Gini index,SI.POV.GINI,1960,NaN
1,Africa Eastern and Southern,AFE,Gini index,SI.POV.GINI,1960,NaN
2,Afghanistan,AFG,Gini index,SI.POV.GINI,1960,NaN
3,Africa Western and Central,AFW,Gini index,SI.POV.GINI,1960,NaN
4,Angola,AGO,Gini index,SI.POV.GINI,1960,NaN
...,...,...,...,...,...,...
16753,Kosovo,XKX,Gini index,SI.POV.GINI,2022,NaN
16754,"Yemen, Rep.",YEM,Gini index,SI.POV.GINI,2022,NaN
16755,South Africa,ZAF,Gini index,SI.POV.GINI,2022,NaN
16756,Zambia,ZMB,Gini index,SI.POV.GINI,2022,NaN


Again chaging data type to integer and then filtering out years before 2010.


In [27]:
gini_years_only['Year'] = pd.to_numeric(gini_years_only['Year'], errors='coerce')
gini_years_only.dtypes

/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/2558772313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gini_years_only['Year'] = pd.to_numeric(gini_years_only['Year'], errors='coerce')


Country Name       object
Country Code       object
Indicator Name     object
Indicator Code     object
Year                int64
Value             float64
dtype: object

In [28]:
gini_relevant_years = gini_years_only[(gini_years_only['Year'] >= 2010)]
gini_relevant_years

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
13300,Aruba,ABW,Gini index,SI.POV.GINI,2010,NaN
13301,Africa Eastern and Southern,AFE,Gini index,SI.POV.GINI,2010,NaN
13302,Afghanistan,AFG,Gini index,SI.POV.GINI,2010,NaN
13303,Africa Western and Central,AFW,Gini index,SI.POV.GINI,2010,NaN
13304,Angola,AGO,Gini index,SI.POV.GINI,2010,NaN
...,...,...,...,...,...,...
16753,Kosovo,XKX,Gini index,SI.POV.GINI,2022,NaN
16754,"Yemen, Rep.",YEM,Gini index,SI.POV.GINI,2022,NaN
16755,South Africa,ZAF,Gini index,SI.POV.GINI,2022,NaN
16756,Zambia,ZMB,Gini index,SI.POV.GINI,2022,NaN


In [29]:
merged2_df = pd.merge(internet_gni_df, gini_relevant_years[['Country Code', 'Year', 'Value']], how='outer', left_on=['ISO', 'Year'], right_on=['Country Code', 'Year'])
merged2_df = merged2_df.rename(columns={'Value': 'GINI Index'})
merged2_df

,Country,Region,ISO,Year,"Individuals using the Internet, total (%)",Country Code_x,GNI per capita,Country Code_y,GINI Index
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,AFG,520.0,AFG,NaN
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,AFG,550.0,AFG,NaN
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,AFG,650.0,AFG,NaN
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,AFG,680.0,AFG,NaN
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,AFG,650.0,AFG,NaN
...,...,...,...,...,...,...,...,...,...
3469,NaN,NaN,NaN,2022,NaN,NaN,NaN,UMC,NaN
3470,NaN,NaN,NaN,2022,NaN,NaN,NaN,VGB,NaN
3471,NaN,NaN,NaN,2022,NaN,NaN,NaN,VIR,NaN
3472,NaN,NaN,NaN,2022,NaN,NaN,NaN,WLD,NaN


Again checking and then removing aggregate results for regions etc.

In [30]:
codes_to_drop2 = []
country_codes2 = merged2_df['Country Code_y'].tolist()
iso_codes2 = merged2_df['ISO'].tolist()

for code in country_codes2:
    if code not in iso_codes2:
        codes_to_drop2.append(code)

country_code_country_dict2 = dict(zip(gini_relevant_years['Country Code'], gini_relevant_years['Country Name']))

filtered_dict2 = {key: value for key, value in country_code_country_dict2.items() if key in codes_to_drop2}
filtered_dict2

{'ABW': 'Aruba',
 'AFE': 'Africa Eastern and Southern',
 'AFW': 'Africa Western and Central',
 'ARB': 'Arab World',
 'ASM': 'American Samoa',
 'BMU': 'Bermuda',
 'CEB': 'Central Europe and the Baltics',
 'CHI': 'Channel Islands',
 'CSS': 'Caribbean small states',
 'CUW': 'Curacao',
 'CYM': 'Cayman Islands',
 'EAP': 'East Asia & Pacific (excluding high income)',
 'EAR': 'Early-demographic dividend',
 'EAS': 'East Asia & Pacific',
 'ECA': 'Europe & Central Asia (excluding high income)',
 'ECS': 'Europe & Central Asia',
 'EMU': 'Euro area',
 'EUU': 'European Union',
 'FCS': 'Fragile and conflict affected situations',
 'FRO': 'Faroe Islands',
 'GIB': 'Gibraltar',
 'GRL': 'Greenland',
 'GUM': 'Guam',
 'HIC': 'High income',
 'HPC': 'Heavily indebted poor countries (HIPC)',
 'IBD': 'IBRD only',
 'IBT': 'IDA & IBRD total',
 'IDA': 'IDA total',
 'IDB': 'IDA blend',
 'IDX': 'IDA only',
 'IMN': 'Isle of Man',
 'INX': 'Not classified',
 'LAC': 'Latin America & Caribbean (excluding high income)',
 

In [31]:
internet_gni_gini_df = merged2_df[~merged2_df['Country Code_y'].isin(codes_to_drop2)]
internet_gni_gini_df

,Country,Region,ISO,Year,"Individuals using the Internet, total (%)",Country Code_x,GNI per capita,Country Code_y,GINI Index
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,AFG,520.0,AFG,NaN
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,AFG,550.0,AFG,NaN
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,AFG,650.0,AFG,NaN
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,AFG,680.0,AFG,NaN
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,AFG,650.0,AFG,NaN
...,...,...,...,...,...,...,...,...,...
2533,Zimbabwe,Africa,ZWE,2020,29.298565,ZWE,1460.0,ZWE,NaN
2534,Zimbabwe,Africa,ZWE,2021,34.813971,ZWE,1530.0,ZWE,NaN
2535,Zimbabwe,Africa,ZWE,2022,NaN,ZWE,1500.0,ZWE,NaN
2536,Eritrea,Africa,ERI,2018,NaN,ERI,NaN,ERI,NaN


In [32]:
columns_to_drop = ['Country Code_x', 'Country Code_y']
internet_gni_gini_df.drop(columns=columns_to_drop, inplace=True, axis=1)

/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/2539365989.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  internet_gni_gini_df.drop(columns=columns_to_drop, inplace=True, axis=1)


In [33]:
internet_gni_gini_df.shape

(2538, 7)

At the end of this section the dataset is called internet_gni_gini_df

## GDP Data Cleaning <a name="gdpcleaning"></a>
Pulling in and setting up the data from the world bank dataset on GDP by country by year.

In [34]:
gdp_initial_data = pd.read_csv("data/API_NY.GDP.PCAP.CD_DS2_en_csv_v2_5994720.csv", skiprows=4)
gdp_initial_data

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Unnamed: 67
0,Aruba,ABW,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6283.001443,7567.253642,9274.514156,10767.396220,11638.733706,12850.215712,13657.670644,14970.152342,16675.278488,17140.433369,17375.225306,18713.425388,19742.316739,19833.826746,21023.157513,20913.299497,21377.095185,22050.830932,24104.646177,24975.673257,25833.445623,27665.426465,29011.559245,25739.137251,24452.928363,26044.435933,25609.955724,26515.678080,26942.307976,28421.386493,28451.273745,29326.708058,30918.515218,31902.762582,24487.863569,29342.100730,NaN,NaN
1,Africa Eastern and Southern,AFE,GDP per capita (current US$),NY.GDP.PCAP.CD,161.638982,161.111657,170.538537,198.038919,178.019217,196.818756,207.945574,210.149894,223.040659,248.448220,258.760362,277.028107,290.929486,369.782388,441.214102,455.808233,439.627553,484.274983,533.768074,598.516558,741.305705,739.098653,685.023457,694.537147,617.375482,510.414257,554.458043,656.115225,699.348882,724.754485,819.844296,869.131599,752.716098,723.802778,695.696584,764.788395,742.428029,761.231441,698.562841,671.862171,709.061010,630.198869,630.479082,816.437662,989.220845,1124.220303,1230.194793,1374.086248,1433.258304,1417.130587,1649.639146,1799.623009,1765.250073,1736.222507,1724.534381,1545.559084,1436.094404,1613.668488,1550.111762,1499.954437,1353.927527,1539.165543,1622.391720,NaN
2,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,62.369375,62.443703,60.950364,82.021738,85.511073,105.243196,143.103233,167.165675,134.012768,134.250360,162.642175,166.224831,141.365322,149.744068,181.598776,194.669036,205.674097,232.778699,255.045622,284.755532,291.649791,311.853617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,183.532775,200.462376,221.657662,255.055107,274.000656,375.078261,387.849288,443.845151,554.594735,621.912414,663.141053,651.987862,628.146804,592.476165,520.251955,530.149863,502.057099,500.522981,516.866797,363.674087,NaN,NaN
3,Africa Western and Central,AFW,GDP per capita (current US$),NY.GDP.PCAP.CD,107.423775,112.503866,118.197351,122.774120,131.163760,137.809818,143.601581,128.049650,129.123648,143.145552,194.327390,168.385279,199.319750,244.905282,331.581835,375.628580,441.401165,451.997204,479.177640,579.620309,712.426374,1301.957532,1122.290300,806.591433,650.459671,645.828620,580.908731,580.461595,558.447755,507.740610,591.989608,556.337330,545.465986,443.597693,377.211747,460.825071,521.416803,513.050543,511.494584,526.248227,522.772772,535.897917,621.862505,700.443384,843.989814,1003.436618,1245.822948,1420.840336,1685.371170,1467.241236,1679.646727,1860.943928,1957.519614,2153.766138,2247.857492,1880.750778,1646.972399,1588.581005,1732.816544,1810.538006,1686.997899,1765.965307,1790.348800,NaN
4,Angola,AGO,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,711.941169,643.052851,620.363110,623.533933,637.519759,757.632389,684.489277,755.972445,792.914088,891.803205,949.349841,1038.914019,1196.464962,847.653799,251.856499,399.735558,523.310909,514.309887,423.393453,387.689415,556.884244,527.464118,872.657638,982.805590,1254.696126,1900.723817,2597.963585,3121.348735,4081.717497,3123.698898,3496.784796,4511.153227,4962.552072,5101.983876,5059.080441,3100.830685,1709.515534,2283.214233,2487.500996,2142.238757,1502.950754,1903.717405,2998.501158,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [35]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
gdp_initial_data.isnull().sum()

Country Name        0
Country Code        0
Indicator Name      0
Indicator Code      0
1960              134
1961              132
1962              130
1963              130
1964              130
1965              119
1966              116
1967              113
1968              108
1969              108
1970               99
1971               96
1972               96
1973               96
1974               95
1975               93
1976               92
1977               89
1978               90
1979               89
1980               75
1981               72
1982               71
1983               70
1984               69
1985               67
1986               65
1987               61
1988               56
1989               56
1990               40
1991               40
1992               39
1993               37
1994               35
1995               26
1996               26
1997               26
1998               24
1999               23
2000               19
2001      

In [36]:
pd.set_option('display.max_rows', 10)

gdp_unpivot_dataset = pd.melt(gdp_initial_data, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
                       var_name='Year', value_name='Value')

gdp_unpivot_dataset

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
0,Aruba,ABW,GDP per capita (current US$),NY.GDP.PCAP.CD,1960,NaN
1,Africa Eastern and Southern,AFE,GDP per capita (current US$),NY.GDP.PCAP.CD,1960,161.638982
2,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,1960,62.369375
3,Africa Western and Central,AFW,GDP per capita (current US$),NY.GDP.PCAP.CD,1960,107.423775
4,Angola,AGO,GDP per capita (current US$),NY.GDP.PCAP.CD,1960,NaN
...,...,...,...,...,...,...
17019,Kosovo,XKX,GDP per capita (current US$),NY.GDP.PCAP.CD,Unnamed: 67,NaN
17020,"Yemen, Rep.",YEM,GDP per capita (current US$),NY.GDP.PCAP.CD,Unnamed: 67,NaN
17021,South Africa,ZAF,GDP per capita (current US$),NY.GDP.PCAP.CD,Unnamed: 67,NaN
17022,Zambia,ZMB,GDP per capita (current US$),NY.GDP.PCAP.CD,Unnamed: 67,NaN


In [37]:
gdp_years_only = gdp_unpivot_dataset[(gini_unpivot_dataset['Year'] != 'Unnamed: 67')]

gdp_years_only.loc[:, 'Year'] = gdp_years_only['Year'].astype(int)

gdp_relevant_years = gdp_years_only[(gdp_years_only['Year'] >= 2010)]
gdp_relevant_years

,Country Name,Country Code,Indicator Name,Indicator Code,Year,Value
13300,Aruba,ABW,GDP per capita (current US$),NY.GDP.PCAP.CD,2010,24452.928363
13301,Africa Eastern and Southern,AFE,GDP per capita (current US$),NY.GDP.PCAP.CD,2010,1649.639146
13302,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,2010,554.594735
13303,Africa Western and Central,AFW,GDP per capita (current US$),NY.GDP.PCAP.CD,2010,1679.646727
13304,Angola,AGO,GDP per capita (current US$),NY.GDP.PCAP.CD,2010,3496.784796
...,...,...,...,...,...,...
16753,Kosovo,XKX,GDP per capita (current US$),NY.GDP.PCAP.CD,2022,5351.439542
16754,"Yemen, Rep.",YEM,GDP per capita (current US$),NY.GDP.PCAP.CD,2022,676.928385
16755,South Africa,ZAF,GDP per capita (current US$),NY.GDP.PCAP.CD,2022,6776.480077
16756,Zambia,ZMB,GDP per capita (current US$),NY.GDP.PCAP.CD,2022,1487.907764


In [38]:
merged3_df = pd.merge(internet_gni_gini_df, gdp_relevant_years[['Country Code', 'Year', 'Value']], how='outer', left_on=['ISO', 'Year'], right_on=['Country Code', 'Year'])
merged3_df = merged3_df.rename(columns={'Value': 'GDP Per Capita'})
merged3_df

,Country,Region,ISO,Year,"Individuals using the Internet, total (%)",GNI per capita,GINI Index,Country Code,GDP Per Capita
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,520.0,NaN,AFG,554.594735
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,550.0,NaN,AFG,621.912414
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,650.0,NaN,AFG,663.141053
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,680.0,NaN,AFG,651.987862
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,650.0,NaN,AFG,628.146804
...,...,...,...,...,...,...,...,...,...
3469,NaN,NaN,NaN,2022,NaN,NaN,NaN,UMC,10794.931857
3470,NaN,NaN,NaN,2022,NaN,NaN,NaN,VGB,NaN
3471,NaN,NaN,NaN,2022,NaN,NaN,NaN,VIR,NaN
3472,NaN,NaN,NaN,2022,NaN,NaN,NaN,WLD,12647.480789


In [39]:
codes_to_drop3 = []
country_codes3 = merged3_df['Country Code'].tolist()
iso_codes3 = merged3_df['ISO'].tolist()

for code in country_codes3:
    if code not in iso_codes3:
        codes_to_drop3.append(code)

country_code_country_dict3 = dict(zip(gdp_relevant_years['Country Code'], gdp_relevant_years['Country Name']))

filtered_dict3 = {key: value for key, value in country_code_country_dict3.items() if key in codes_to_drop3}
filtered_dict3

{'ABW': 'Aruba',
 'AFE': 'Africa Eastern and Southern',
 'AFW': 'Africa Western and Central',
 'ARB': 'Arab World',
 'ASM': 'American Samoa',
 'BMU': 'Bermuda',
 'CEB': 'Central Europe and the Baltics',
 'CHI': 'Channel Islands',
 'CSS': 'Caribbean small states',
 'CUW': 'Curacao',
 'CYM': 'Cayman Islands',
 'EAP': 'East Asia & Pacific (excluding high income)',
 'EAR': 'Early-demographic dividend',
 'EAS': 'East Asia & Pacific',
 'ECA': 'Europe & Central Asia (excluding high income)',
 'ECS': 'Europe & Central Asia',
 'EMU': 'Euro area',
 'EUU': 'European Union',
 'FCS': 'Fragile and conflict affected situations',
 'FRO': 'Faroe Islands',
 'GIB': 'Gibraltar',
 'GRL': 'Greenland',
 'GUM': 'Guam',
 'HIC': 'High income',
 'HPC': 'Heavily indebted poor countries (HIPC)',
 'IBD': 'IBRD only',
 'IBT': 'IDA & IBRD total',
 'IDA': 'IDA total',
 'IDB': 'IDA blend',
 'IDX': 'IDA only',
 'IMN': 'Isle of Man',
 'INX': 'Not classified',
 'LAC': 'Latin America & Caribbean (excluding high income)',
 

In [40]:
internet_gni_gini_gdp_df = merged3_df[~merged3_df['Country Code'].isin(codes_to_drop3)]
internet_gni_gini_gdp_df

,Country,Region,ISO,Year,"Individuals using the Internet, total (%)",GNI per capita,GINI Index,Country Code,GDP Per Capita
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,520.0,NaN,AFG,554.594735
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,550.0,NaN,AFG,621.912414
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,650.0,NaN,AFG,663.141053
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,680.0,NaN,AFG,651.987862
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,650.0,NaN,AFG,628.146804
...,...,...,...,...,...,...,...,...,...
2533,Zimbabwe,Africa,ZWE,2020,29.298565,1460.0,NaN,ZWE,1372.696674
2534,Zimbabwe,Africa,ZWE,2021,34.813971,1530.0,NaN,ZWE,1773.920411
2535,Zimbabwe,Africa,ZWE,2022,NaN,1500.0,NaN,ZWE,1266.996031
2536,Eritrea,Africa,ERI,2018,NaN,NaN,NaN,ERI,NaN


In [41]:
internet_gni_gini_gdp_df.drop(columns=['Country Code'], axis=1, inplace=True)

/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/3464669002.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  internet_gni_gini_gdp_df.drop(columns=['Country Code'], axis=1, inplace=True)


In [42]:
internet_gni_gini_gdp_df.shape

(2538, 8)

Final dataset after this section is called: internet_gni_gini_gdp_df

## Happiness Index Cleaning <a name="happinesindexcleaning"></a>
Pulling in and setting up the data on the Happiness Index by country by year.

In [43]:
happiness_index = pd.read_csv("data/World Happiness Index by Reports 2013-2023 no nulls.csv")
happiness_index

,Country,Year,Index,Rank
0,Afghanistan,2013,4.040,143
1,Afghanistan,2015,3.575,153
2,Afghanistan,2016,3.360,154
3,Afghanistan,2017,3.794,141
4,Afghanistan,2018,3.632,145
...,...,...,...,...
1518,Zimbabwe,2019,3.663,146
1519,Zimbabwe,2020,3.299,151
1520,Zimbabwe,2021,3.145,148
1521,Zimbabwe,2022,2.995,144


This dataset doesn't use ISO codes which will cause problems if there are different spellings. I found some datasets on github which try to solve this issue but linking different spellings to the world bank spelling and hence the ISO code, will use that to add an ISO column to the initial dataset.

In [44]:
pd.set_option('display.max_rows', 10)
country_names = pd.read_csv("data/any_name_to_wb_name.csv")
iso_codes = pd.read_csv("data/iso3_to_wb_name.csv")
country_names

,any,wb_name
0,Afghanistan,Afghanistan
1,"Afghanistan, islamic republic of",Afghanistan
2,Aland Islands,NaN
3,Aland,NaN
4,Albania,Albania
...,...,...
406,Yemen,"Yemen, Rep."
407,"Yemen, republic of","Yemen, Rep."
408,"Yemen, Rep.","Yemen, Rep."
409,Zambia,Zambia


In [45]:
hap_merge_names = pd.merge(happiness_index, country_names, how="left", left_on="Country", right_on="any")
list_missing = hap_merge_names[hap_merge_names["wb_name"].isnull()]
countries = list_missing["Country"].unique()
countries

array(['Congo Brazzaville', 'Congo Kinshasa', 'Czechia', 'Eswatini',
       'North Cyprus', 'North Macedonia', 'Palestine', 'Somaliland',
       'Turkiye'], dtype=object)

Some of these I can just add to the csv with the spelling / name the world bank references and others are contested territories which won't appear in the other datasets so I will just drop before the merger

In [46]:
new_rows = [
    {'any': 'Congo Brazzaville', 'wb_name': "Congo, Rep."},
    {'any': 'Congo Kinshasa', 'wb_name': "Congo, Dem. Rep."},
    {'any': 'Czechia', 'wb_name': "Czech Republic"},
    {'any': 'Eswatini', 'wb_name': "Swaziland"},
    {'any': 'North Macedonia', 'wb_name': "Macedonia, FYR"},
    {'any': 'Palestine', 'wb_name': "West Bank and Gaza"},
    {'any': 'Turkiye', 'wb_name': "Turkey"}
]

new_rows_df = pd.DataFrame(new_rows)

country_names = pd.concat([country_names, new_rows_df], ignore_index=True)

Now rerunning the merger and checking empty rows

In [47]:
hap_merge_names = pd.merge(happiness_index, country_names, how="left", left_on="Country", right_on="any")
list_missing = hap_merge_names[hap_merge_names["wb_name"].isnull()]
countries = list_missing["Country"].unique()
countries

array(['North Cyprus', 'Somaliland'], dtype=object)

In [48]:
hap_merge_names = hap_merge_names[(hap_merge_names['Country'] != 'North Cyprus') & (hap_merge_names['Country'] != 'Somaliland')]

In [49]:
hap_merge_names

,Country,Year,Index,Rank,any,wb_name
0,Afghanistan,2013,4.040,143,Afghanistan,Afghanistan
1,Afghanistan,2015,3.575,153,Afghanistan,Afghanistan
2,Afghanistan,2016,3.360,154,Afghanistan,Afghanistan
3,Afghanistan,2017,3.794,141,Afghanistan,Afghanistan
4,Afghanistan,2018,3.632,145,Afghanistan,Afghanistan
...,...,...,...,...,...,...
1518,Zimbabwe,2019,3.663,146,Zimbabwe,Zimbabwe
1519,Zimbabwe,2020,3.299,151,Zimbabwe,Zimbabwe
1520,Zimbabwe,2021,3.145,148,Zimbabwe,Zimbabwe
1521,Zimbabwe,2022,2.995,144,Zimbabwe,Zimbabwe


In [50]:
iso_codes.rename(columns={'country': 'Country WB Name'}, inplace=True)
iso_codes

,iso3,Country WB Name
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,AIA,Anguilla
4,ALB,Albania
...,...,...
216,WSM,Samoa
217,YEM,"Yemen, Rep."
218,ZAF,South Africa
219,ZMB,Zambia


In [51]:
hap_merge_iso = pd.merge(hap_merge_names, iso_codes, how="left", left_on="wb_name", right_on="Country WB Name")
hap_merge_iso

,Country,Year,Index,Rank,any,wb_name,iso3,Country WB Name
0,Afghanistan,2013,4.040,143,Afghanistan,Afghanistan,AFG,Afghanistan
1,Afghanistan,2015,3.575,153,Afghanistan,Afghanistan,AFG,Afghanistan
2,Afghanistan,2016,3.360,154,Afghanistan,Afghanistan,AFG,Afghanistan
3,Afghanistan,2017,3.794,141,Afghanistan,Afghanistan,AFG,Afghanistan
4,Afghanistan,2018,3.632,145,Afghanistan,Afghanistan,AFG,Afghanistan
...,...,...,...,...,...,...,...,...
1526,Zimbabwe,2019,3.663,146,Zimbabwe,Zimbabwe,ZWE,Zimbabwe
1527,Zimbabwe,2020,3.299,151,Zimbabwe,Zimbabwe,ZWE,Zimbabwe
1528,Zimbabwe,2021,3.145,148,Zimbabwe,Zimbabwe,ZWE,Zimbabwe
1529,Zimbabwe,2022,2.995,144,Zimbabwe,Zimbabwe,ZWE,Zimbabwe


In [52]:
hap_merge_iso.drop_duplicates(subset=['Country', 'Year', 'Index'], inplace=True)
hap_merge_iso

,Country,Year,Index,Rank,any,wb_name,iso3,Country WB Name
0,Afghanistan,2013,4.040,143,Afghanistan,Afghanistan,AFG,Afghanistan
1,Afghanistan,2015,3.575,153,Afghanistan,Afghanistan,AFG,Afghanistan
2,Afghanistan,2016,3.360,154,Afghanistan,Afghanistan,AFG,Afghanistan
3,Afghanistan,2017,3.794,141,Afghanistan,Afghanistan,AFG,Afghanistan
4,Afghanistan,2018,3.632,145,Afghanistan,Afghanistan,AFG,Afghanistan
...,...,...,...,...,...,...,...,...
1526,Zimbabwe,2019,3.663,146,Zimbabwe,Zimbabwe,ZWE,Zimbabwe
1527,Zimbabwe,2020,3.299,151,Zimbabwe,Zimbabwe,ZWE,Zimbabwe
1528,Zimbabwe,2021,3.145,148,Zimbabwe,Zimbabwe,ZWE,Zimbabwe
1529,Zimbabwe,2022,2.995,144,Zimbabwe,Zimbabwe,ZWE,Zimbabwe


In [53]:
hap_merge_iso.rename(columns={'iso3': 'ISO'}, inplace=True)
hap_merge_iso

,Country,Year,Index,Rank,any,wb_name,ISO,Country WB Name
0,Afghanistan,2013,4.040,143,Afghanistan,Afghanistan,AFG,Afghanistan
1,Afghanistan,2015,3.575,153,Afghanistan,Afghanistan,AFG,Afghanistan
2,Afghanistan,2016,3.360,154,Afghanistan,Afghanistan,AFG,Afghanistan
3,Afghanistan,2017,3.794,141,Afghanistan,Afghanistan,AFG,Afghanistan
4,Afghanistan,2018,3.632,145,Afghanistan,Afghanistan,AFG,Afghanistan
...,...,...,...,...,...,...,...,...
1526,Zimbabwe,2019,3.663,146,Zimbabwe,Zimbabwe,ZWE,Zimbabwe
1527,Zimbabwe,2020,3.299,151,Zimbabwe,Zimbabwe,ZWE,Zimbabwe
1528,Zimbabwe,2021,3.145,148,Zimbabwe,Zimbabwe,ZWE,Zimbabwe
1529,Zimbabwe,2022,2.995,144,Zimbabwe,Zimbabwe,ZWE,Zimbabwe


In [54]:
merged4_df = pd.merge(internet_gni_gini_gdp_df, hap_merge_iso[["Index", "ISO", "Year", "Country"]], how='outer', left_on=['ISO', 'Year'], right_on=['ISO', 'Year'])
merged4_df = merged4_df.rename(columns={'Index': 'Happiness Index'})
merged4_df

,Country_x,Region,ISO,Year,"Individuals using the Internet, total (%)",GNI per capita,GINI Index,GDP Per Capita,Happiness Index,Country_y
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,520.0,NaN,554.594735,NaN,NaN
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,550.0,NaN,621.912414,NaN,NaN
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,650.0,NaN,663.141053,NaN,NaN
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,680.0,NaN,651.987862,4.040,Afghanistan
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,650.0,NaN,628.146804,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2698,NaN,NaN,UZB,2023,NaN,NaN,NaN,NaN,6.014,Uzbekistan
2699,NaN,NaN,VEN,2023,NaN,NaN,NaN,NaN,5.211,Venezuela
2700,NaN,NaN,VNM,2023,NaN,NaN,NaN,NaN,5.763,Vietnam
2701,NaN,NaN,ZMB,2023,NaN,NaN,NaN,NaN,3.982,Zambia


In [55]:
merged4_df.isna().sum()

Country_x                                     165
Region                                        165
ISO                                             0
Year                                            0
Individuals using the Internet, total (%)     400
GNI per capita                                296
GINI Index                                   1821
GDP Per Capita                                236
Happiness Index                              1192
Country_y                                    1192
dtype: int64

Need to fill the countries / rows which were added through the happiness index dataset and are missing regions and country names in the main country column

In [56]:
country_info_dict = dict(zip(internet_gni_gini_gdp_df['ISO'], zip(internet_gni_gini_gdp_df['Country'], internet_gni_gini_gdp_df['Region'])))
country_info_dict

for index, row in merged4_df.iterrows():
    iso = row["ISO"]
    if iso in country_info_dict:
        country_name, region = country_info_dict[iso]
        if pd.isnull(row["Country_x"]):
            merged4_df.at[index, "Country_x"] = country_name
        if pd.isnull(row["Region"]):
            merged4_df.at[index, "Region"] = region
        if pd.isnull(row["ISO"]):
            merged4_df.at[index, "ISO"] = iso_code

merged4_df

,Country_x,Region,ISO,Year,"Individuals using the Internet, total (%)",GNI per capita,GINI Index,GDP Per Capita,Happiness Index,Country_y
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,520.0,NaN,554.594735,NaN,NaN
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,550.0,NaN,621.912414,NaN,NaN
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,650.0,NaN,663.141053,NaN,NaN
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,680.0,NaN,651.987862,4.040,Afghanistan
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,650.0,NaN,628.146804,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2698,Uzbekistan,CIS,UZB,2023,NaN,NaN,NaN,NaN,6.014,Uzbekistan
2699,Venezuela,The Americas,VEN,2023,NaN,NaN,NaN,NaN,5.211,Venezuela
2700,Viet Nam,Asia & Pacific,VNM,2023,NaN,NaN,NaN,NaN,5.763,Vietnam
2701,Zambia,Africa,ZMB,2023,NaN,NaN,NaN,NaN,3.982,Zambia


In [57]:
merged4_df.isna().sum()

Country_x                                      31
Region                                         31
ISO                                             0
Year                                            0
Individuals using the Internet, total (%)     400
GNI per capita                                296
GINI Index                                   1821
GDP Per Capita                                236
Happiness Index                              1192
Country_y                                    1192
dtype: int64

There are still 31 lines for countries which weren't matched with previous ISO codes

In [58]:
merged4_df[merged4_df["Country_x"].isnull()]

,Country_x,Region,ISO,Year,"Individuals using the Internet, total (%)",GNI per capita,GINI Index,GDP Per Capita,Happiness Index,Country_y
2603,NaN,NaN,KOS,2013,NaN,NaN,NaN,NaN,5.222,Kosovo
2604,NaN,NaN,KOS,2015,NaN,NaN,NaN,NaN,5.589,Kosovo
2605,NaN,NaN,KOS,2016,NaN,NaN,NaN,NaN,5.401,Kosovo
2606,NaN,NaN,KOS,2017,NaN,NaN,NaN,NaN,5.279,Kosovo
2607,NaN,NaN,KOS,2018,NaN,NaN,NaN,NaN,5.662,Kosovo
...,...,...,...,...,...,...,...,...,...,...
2681,NaN,NaN,TWN,2019,NaN,NaN,NaN,NaN,6.446,Taiwan
2682,NaN,NaN,TWN,2020,NaN,NaN,NaN,NaN,6.455,Taiwan
2683,NaN,NaN,TWN,2021,NaN,NaN,NaN,NaN,6.584,Taiwan
2684,NaN,NaN,TWN,2022,NaN,NaN,NaN,NaN,6.512,Taiwan


Again these are countries which don't appear in the world bank data set so we'll need to drop them

In [59]:
merged4_df = merged4_df.dropna(subset=['Country_x', 'Region'], how='any')
merged4_df = merged4_df.drop(columns=['Country_y'])
internet_gni_gini_gdp_hap_df = merged4_df.rename(columns={'Country_x': 'Country'})
internet_gni_gini_gdp_hap_df

,Country,Region,ISO,Year,"Individuals using the Internet, total (%)",GNI per capita,GINI Index,GDP Per Capita,Happiness Index
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,520.0,NaN,554.594735,NaN
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,550.0,NaN,621.912414,NaN
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,650.0,NaN,663.141053,NaN
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,680.0,NaN,651.987862,4.040
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,650.0,NaN,628.146804,NaN
...,...,...,...,...,...,...,...,...,...
2698,Uzbekistan,CIS,UZB,2023,NaN,NaN,NaN,NaN,6.014
2699,Venezuela,The Americas,VEN,2023,NaN,NaN,NaN,NaN,5.211
2700,Viet Nam,Asia & Pacific,VNM,2023,NaN,NaN,NaN,NaN,5.763
2701,Zambia,Africa,ZMB,2023,NaN,NaN,NaN,NaN,3.982


In [60]:
internet_gni_gini_gdp_hap_df.shape

(2672, 9)

At the end of this section the final dataframe is called internet_gni_gini_gdp_hap_df

## Literacy Rates <a name="literacyratescleaning"></a>
Pulling in and setting up the data via an api on the Literacy Rates by country by year.

In [61]:
url = 'http://api.worldbank.org/v2/country/all/indicator/SE.ADT.LITR.ZS?format=json&page='

all_data = []
page = 1

while True:
    response = requests.get(url + str(page))
    data = response.json()

    if data[0]['page'] >= data[0]['pages']:
        break

    all_data.extend(data[1])
    page += 1

print(len(all_data))

16750


In [62]:
df = pd.DataFrame(all_data)
df['date'] = df['date'].astype(int)
df.dtypes

indicator           object
country             object
countryiso3code     object
date                 int64
value              float64
unit                object
obs_status          object
decimal              int64
dtype: object

Getting rid of data before 2010 to match with the remaining datasets

In [63]:
pd.set_option('display.max_rows', 10)
df_reduced = df[df["date"]>=2010]
df_reduced

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2022,72.600403,,,0
1,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2021,72.581161,,,0
2,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,72.785622,,,0
3,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2019,72.634972,,,0
4,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2018,71.392616,,,0
...,...,...,...,...,...,...,...,...
16703,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2014,88.693420,,,0
16704,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2013,NaN,,,0
16705,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2012,NaN,,,0
16706,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2011,83.582710,,,0


Pulling out the name from the dictionary value in the country column

In [64]:
df_reduced['Country'] = df['country'].apply(lambda x: x['value'])
df_reduced

/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/2594793725.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced['Country'] = df['country'].apply(lambda x: x['value'])


,indicator,country,countryiso3code,date,value,unit,obs_status,decimal,Country
0,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2022,72.600403,,,0,Africa Eastern and Southern
1,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2021,72.581161,,,0,Africa Eastern and Southern
2,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,72.785622,,,0,Africa Eastern and Southern
3,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2019,72.634972,,,0,Africa Eastern and Southern
4,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2018,71.392616,,,0,Africa Eastern and Southern
...,...,...,...,...,...,...,...,...,...
16703,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2014,88.693420,,,0,Zimbabwe
16704,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2013,NaN,,,0,Zimbabwe
16705,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2012,NaN,,,0,Zimbabwe
16706,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2011,83.582710,,,0,Zimbabwe


In [65]:
df_reduced.dropna(subset=['value'], inplace=True)
df_reduced.rename(columns={'date': 'Year'}, inplace=True)
df_reduced.rename(columns={'countryiso3code': 'ISO'}, inplace=True)
df_reduced

/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/3283445406.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced.dropna(subset=['value'], inplace=True)
/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/3283445406.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced.rename(columns={'date': 'Year'}, inplace=True)
/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/3283445406.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

,indicator,country,ISO,Year,value,unit,obs_status,decimal,Country
0,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2022,72.600403,,,0,Africa Eastern and Southern
1,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2021,72.581161,,,0,Africa Eastern and Southern
2,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,72.785622,,,0,Africa Eastern and Southern
3,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2019,72.634972,,,0,Africa Eastern and Southern
4,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2018,71.392616,,,0,Africa Eastern and Southern
...,...,...,...,...,...,...,...,...,...
16634,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZM', 'value': 'Zambia'}",ZMB,2020,87.500000,,,0,Zambia
16644,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZM', 'value': 'Zambia'}",ZMB,2010,83.007668,,,0,Zambia
16695,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2022,89.849998,,,0,Zimbabwe
16703,"{'id': 'SE.ADT.LITR.ZS', 'value': 'Literacy ra...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2014,88.693420,,,0,Zimbabwe


In [66]:
merged5_df = pd.merge(internet_gni_gini_gdp_hap_df, df_reduced[['ISO', 'Year', 'value']], on=['ISO', 'Year'], how='left')
merged5_df.rename(columns={'value': 'Literacy'}, inplace=True)
merged5_df

,Country,Region,ISO,Year,"Individuals using the Internet, total (%)",GNI per capita,GINI Index,GDP Per Capita,Happiness Index,Literacy
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,520.0,NaN,554.594735,NaN,NaN
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,550.0,NaN,621.912414,NaN,31.448851
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,650.0,NaN,663.141053,NaN,NaN
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,680.0,NaN,651.987862,4.040,NaN
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,650.0,NaN,628.146804,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2667,Uzbekistan,CIS,UZB,2023,NaN,NaN,NaN,NaN,6.014,NaN
2668,Venezuela,The Americas,VEN,2023,NaN,NaN,NaN,NaN,5.211,NaN
2669,Viet Nam,Asia & Pacific,VNM,2023,NaN,NaN,NaN,NaN,5.763,NaN
2670,Zambia,Africa,ZMB,2023,NaN,NaN,NaN,NaN,3.982,NaN


In [67]:
internet_gni_gini_gdp_hap_lit_df = merged5_df.copy()

In [68]:
internet_gni_gini_gdp_hap_lit_df.shape

(2672, 10)

The final dataset after this section is called internet_gni_gini_gdp_hap_lit_df

## Suicide Rates <a name="suicideratescleaning"></a>
Pulling in and setting up the data via an api on the Suicide Rates by country by year.

In [69]:
url = 'http://api.worldbank.org/v2/country/all/indicator/SH.STA.SUIC.P5?format=json&page='

all_data = []
page = 1

while True:
    response = requests.get(url + str(page))
    data = response.json()

    if data[0]['page'] >= data[0]['pages']:
        break

    all_data.extend(data[1])
    page += 1

print(len(all_data))

16750


In [70]:
df2 = pd.DataFrame(all_data)
df2['date'] = df2['date'].astype(int)
df2.dtypes

indicator           object
country             object
countryiso3code     object
date                 int64
value              float64
unit                object
obs_status          object
decimal              int64
dtype: object

In [71]:
pd.set_option('display.max_rows', 10)
df_reduced2 = df2[df2["date"]>=2010]
df_reduced2

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2022,NaN,,,1
1,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2021,NaN,,,1
2,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,NaN,,,1
3,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2019,8.024906,,,1
4,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2018,8.118032,,,1
...,...,...,...,...,...,...,...,...
16703,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2014,18.200000,,,1
16704,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2013,18.600000,,,1
16705,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2012,19.700000,,,1
16706,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2011,20.700000,,,1


In [72]:
df_reduced2['Country'] = df2['country'].apply(lambda x: x['value'])
df_reduced2

/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/3746184579.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced2['Country'] = df2['country'].apply(lambda x: x['value'])


,indicator,country,countryiso3code,date,value,unit,obs_status,decimal,Country
0,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2022,NaN,,,1,Africa Eastern and Southern
1,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2021,NaN,,,1,Africa Eastern and Southern
2,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,NaN,,,1,Africa Eastern and Southern
3,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2019,8.024906,,,1,Africa Eastern and Southern
4,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2018,8.118032,,,1,Africa Eastern and Southern
...,...,...,...,...,...,...,...,...,...
16703,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2014,18.200000,,,1,Zimbabwe
16704,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2013,18.600000,,,1,Zimbabwe
16705,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2012,19.700000,,,1,Zimbabwe
16706,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2011,20.700000,,,1,Zimbabwe


In [73]:
df_reduced2.dropna(subset=['value'], inplace=True)
df_reduced2.rename(columns={'date': 'Year'}, inplace=True)
df_reduced2.rename(columns={'countryiso3code': 'ISO'}, inplace=True)
df_reduced2

/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/1834253825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced2.dropna(subset=['value'], inplace=True)
/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/1834253825.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced2.rename(columns={'date': 'Year'}, inplace=True)
/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/1834253825.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

,indicator,country,ISO,Year,value,unit,obs_status,decimal,Country
3,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2019,8.024906,,,1,Africa Eastern and Southern
4,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2018,8.118032,,,1,Africa Eastern and Southern
5,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2017,8.232943,,,1,Africa Eastern and Southern
6,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2016,8.328658,,,1,Africa Eastern and Southern
7,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2015,8.489758,,,1,Africa Eastern and Southern
...,...,...,...,...,...,...,...,...,...
16703,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2014,18.200000,,,1,Zimbabwe
16704,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2013,18.600000,,,1,Zimbabwe
16705,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2012,19.700000,,,1,Zimbabwe
16706,"{'id': 'SH.STA.SUIC.P5', 'value': 'Suicide mor...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2011,20.700000,,,1,Zimbabwe


In [74]:
merged6_df = pd.merge(internet_gni_gini_gdp_hap_lit_df, df_reduced2[['ISO', 'Year', 'value']], on=['ISO', 'Year'], how='left')
merged6_df.rename(columns={'value': 'Suicide Rates'}, inplace=True)
merged6_df

,Country,Region,ISO,Year,"Individuals using the Internet, total (%)",GNI per capita,GINI Index,GDP Per Capita,Happiness Index,Literacy,Suicide Rates
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,520.0,NaN,554.594735,NaN,NaN,4.3
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,550.0,NaN,621.912414,NaN,31.448851,4.1
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,650.0,NaN,663.141053,NaN,NaN,4.0
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,680.0,NaN,651.987862,4.040,NaN,4.0
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,650.0,NaN,628.146804,NaN,NaN,3.9
...,...,...,...,...,...,...,...,...,...,...,...
2667,Uzbekistan,CIS,UZB,2023,NaN,NaN,NaN,NaN,6.014,NaN,NaN
2668,Venezuela,The Americas,VEN,2023,NaN,NaN,NaN,NaN,5.211,NaN,NaN
2669,Viet Nam,Asia & Pacific,VNM,2023,NaN,NaN,NaN,NaN,5.763,NaN,NaN
2670,Zambia,Africa,ZMB,2023,NaN,NaN,NaN,NaN,3.982,NaN,NaN


In [75]:
internet_gni_gini_gdp_hap_lit_sui_df = merged6_df.copy()

In [76]:
internet_gni_gini_gdp_hap_lit_sui_df.shape

(2672, 11)

The final dataset after this section is called internet_gni_gini_gdp_hap_lit_sui_df

## CO2 Emissions <a name="co2cleaning"></a>
Pulling in and setting up the data via an api on the CO2 Emissions by country by year.

In [77]:
url = 'http://api.worldbank.org/v2/country/all/indicator/EN.ATM.CO2E.PC?format=json&page='

all_data = []
page = 1

while True:
    response = requests.get(url + str(page))
    data = response.json()

    if data[0]['page'] >= data[0]['pages']:
        break

    all_data.extend(data[1])
    page += 1

print(len(all_data))

16750


In [78]:
df3 = pd.DataFrame(all_data)
df3['date'] = df3['date'].astype(int)
df3.dtypes

indicator           object
country             object
countryiso3code     object
date                 int64
value              float64
unit                object
obs_status          object
decimal              int64
dtype: object

In [79]:
pd.set_option('display.max_rows', 10)
df_reduced3 = df3[df3["date"]>=2010]
df_reduced3

,indicator,country,countryiso3code,date,value,unit,obs_status,decimal
0,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2022,NaN,,,1
1,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2021,NaN,,,1
2,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,0.795420,,,1
3,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2019,0.915294,,,1
4,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2018,0.921453,,,1
...,...,...,...,...,...,...,...,...
16703,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2014,0.866838,,,1
16704,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2013,0.901248,,,1
16705,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2012,0.901214,,,1
16706,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2011,0.871932,,,1


In [80]:
df_reduced3['Country'] = df3['country'].apply(lambda x: x['value'])
df_reduced3

/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/2051446966.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced3['Country'] = df3['country'].apply(lambda x: x['value'])


,indicator,country,countryiso3code,date,value,unit,obs_status,decimal,Country
0,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2022,NaN,,,1,Africa Eastern and Southern
1,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2021,NaN,,,1,Africa Eastern and Southern
2,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,0.795420,,,1,Africa Eastern and Southern
3,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2019,0.915294,,,1,Africa Eastern and Southern
4,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2018,0.921453,,,1,Africa Eastern and Southern
...,...,...,...,...,...,...,...,...,...
16703,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2014,0.866838,,,1,Zimbabwe
16704,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2013,0.901248,,,1,Zimbabwe
16705,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2012,0.901214,,,1,Zimbabwe
16706,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2011,0.871932,,,1,Zimbabwe


In [81]:
df_reduced3.dropna(subset=['value'], inplace=True)
df_reduced3.rename(columns={'date': 'Year'}, inplace=True)
df_reduced3.rename(columns={'countryiso3code': 'ISO'}, inplace=True)
df_reduced3

/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/1901030127.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced3.dropna(subset=['value'], inplace=True)
/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/1901030127.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced3.rename(columns={'date': 'Year'}, inplace=True)
/var/folders/v4/fzvn3vnd4jg427pq437v42j80000gn/T/ipykernel_79599/1901030127.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

,indicator,country,ISO,Year,value,unit,obs_status,decimal,Country
2,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2020,0.795420,,,1,Africa Eastern and Southern
3,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2019,0.915294,,,1,Africa Eastern and Southern
4,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2018,0.921453,,,1,Africa Eastern and Southern
5,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2017,0.933874,,,1,Africa Eastern and Southern
6,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZH', 'value': 'Africa Eastern and Sout...",AFE,2016,0.941337,,,1,Africa Eastern and Southern
...,...,...,...,...,...,...,...,...,...
16703,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2014,0.866838,,,1,Zimbabwe
16704,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2013,0.901248,,,1,Zimbabwe
16705,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2012,0.901214,,,1,Zimbabwe
16706,"{'id': 'EN.ATM.CO2E.PC', 'value': 'CO2 emissio...","{'id': 'ZW', 'value': 'Zimbabwe'}",ZWE,2011,0.871932,,,1,Zimbabwe


In [82]:
merged7_df = pd.merge(internet_gni_gini_gdp_hap_lit_sui_df, df_reduced3[['ISO', 'Year', 'value']], on=['ISO', 'Year'], how='left')
merged7_df.rename(columns={'value': 'CO2 Emissions'}, inplace=True)
merged7_df

,Country,Region,ISO,Year,"Individuals using the Internet, total (%)",GNI per capita,GINI Index,GDP Per Capita,Happiness Index,Literacy,Suicide Rates,CO2 Emissions
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,520.0,NaN,554.594735,NaN,NaN,4.3,0.304230
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,550.0,NaN,621.912414,NaN,31.448851,4.1,0.408965
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,650.0,NaN,663.141053,NaN,NaN,4.0,0.335061
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,680.0,NaN,651.987862,4.040,NaN,4.0,0.298088
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,650.0,NaN,628.146804,NaN,NaN,3.9,0.283692
...,...,...,...,...,...,...,...,...,...,...,...,...
2667,Uzbekistan,CIS,UZB,2023,NaN,NaN,NaN,NaN,6.014,NaN,NaN,NaN
2668,Venezuela,The Americas,VEN,2023,NaN,NaN,NaN,NaN,5.211,NaN,NaN,NaN
2669,Viet Nam,Asia & Pacific,VNM,2023,NaN,NaN,NaN,NaN,5.763,NaN,NaN,NaN
2670,Zambia,Africa,ZMB,2023,NaN,NaN,NaN,NaN,3.982,NaN,NaN,NaN


In [83]:
internet_gni_gini_gdp_hap_lit_sui_CO2_df = merged7_df.copy()

In [84]:
internet_gni_gini_gdp_hap_lit_sui_CO2_df.shape

(2672, 12)

After this section the final dataset is called internet_gni_gini_gdp_hap_lit_sui_CO2_df

## Internet Price <a name="ipcleaning"></a>

In [85]:
ip_file_path = 'data/internet_price_2021.csv'
ip_df = pd.read_csv(ip_file_path)

# Melt the dataset to convert columns with years into rows
melted = ip_df.melt(id_vars=['City', 'Region', 'Country'], var_name='Year', value_name='Internet_Price')

# Remove rows where 'Internet_Price' is 0 (assuming these are missing values)
melted = melted[melted['Internet_Price'] != 0]

melted.drop(['City', 'Region'], axis=1, inplace=True)

melted['Year'] = melted['Year'].str.split(', ').str[1]
melted['Year'] = melted['Year'].astype(int)

median_ip = melted.groupby(['Country', 'Year'], as_index=False)['Internet_Price'].median()
median_ip = median_ip.sort_values(by=['Year', 'Country'])
median_ip.reset_index(drop=True, inplace=True)

median_ip

,Country,Year,Internet_Price
0,Argentina,2010,34.594820
1,Australia,2010,42.372881
2,Austria,2010,43.209877
3,Bosnia and Herzegovina,2010,21.399177
4,Brazil,2010,34.340659
...,...,...,...
1018,Uruguay,2020,31.760326
1019,Uzbekistan,2020,33.555714
1020,Venezuela,2020,9.112500
1021,Vietnam,2020,10.573918


Now we need to add ISO codes before merging with the main database to avoid mismatches due to different country names.

In [86]:
country_names = pd.read_csv("data/any_name_to_wb_name.csv")
iso_codes = pd.read_csv("data/iso3_to_wb_name.csv")
country_names

,any,wb_name
0,Afghanistan,Afghanistan
1,"Afghanistan, islamic republic of",Afghanistan
2,Aland Islands,NaN
3,Aland,NaN
4,Albania,Albania
...,...,...
406,Yemen,"Yemen, Rep."
407,"Yemen, republic of","Yemen, Rep."
408,"Yemen, Rep.","Yemen, Rep."
409,Zambia,Zambia


In [87]:
ip_merge_names = pd.merge(median_ip, country_names, how="left", left_on="Country", right_on="any")
list_missing = ip_merge_names[ip_merge_names["wb_name"].isnull()]
countries = list_missing["Country"].unique()
countries

array(["People's Republic of China", 'North Macedonia'], dtype=object)

In [88]:
new_rows = [
    {'any': "People's Republic of China", 'wb_name': 'China'},
    {'any': 'North Macedonia', 'wb_name': "Macedonia, FYR"}
]

new_rows_df = pd.DataFrame(new_rows)

country_names = pd.concat([country_names, new_rows_df], ignore_index=True)

Rerunning the merger now to check that all countries have been matched to World Bank spelling of names:

In [89]:
ip_merge_names = pd.merge(median_ip, country_names, how="left", left_on="Country", right_on="any")
list_missing = ip_merge_names[ip_merge_names["wb_name"].isnull()]
countries = list_missing["Country"].unique()
countries

array([], dtype=object)

In [90]:
iso_codes.rename(columns={'country': 'Country WB Name'}, inplace=True)
iso_codes

,iso3,Country WB Name
0,ABW,Aruba
1,AFG,Afghanistan
2,AGO,Angola
3,AIA,Anguilla
4,ALB,Albania
...,...,...
216,WSM,Samoa
217,YEM,"Yemen, Rep."
218,ZAF,South Africa
219,ZMB,Zambia


In [91]:
ip_merge_iso = pd.merge(ip_merge_names, iso_codes, how="left", left_on="wb_name", right_on="Country WB Name")
ip_merge_iso.rename(columns={'iso3': 'ISO'}, inplace=True)
ip_merge_iso

,Country,Year,Internet_Price,any,wb_name,ISO,Country WB Name
0,Argentina,2010,34.594820,Argentina,Argentina,ARG,Argentina
1,Australia,2010,42.372881,Australia,Australia,AUS,Australia
2,Austria,2010,43.209877,Austria,Austria,AUT,Austria
3,Bosnia and Herzegovina,2010,21.399177,Bosnia and Herzegovina,Bosnia and Herzegovina,BIH,Bosnia and Herzegovina
4,Brazil,2010,34.340659,Brazil,Brazil,BRA,Brazil
...,...,...,...,...,...,...,...
1018,Uruguay,2020,31.760326,Uruguay,Uruguay,URY,Uruguay
1019,Uzbekistan,2020,33.555714,Uzbekistan,Uzbekistan,UZB,Uzbekistan
1020,Venezuela,2020,9.112500,Venezuela,"Venezuela, RB",VEN,"Venezuela, RB"
1021,Vietnam,2020,10.573918,Vietnam,Vietnam,VNM,Vietnam


In [92]:
# Merge the datasets
merged = pd.merge(internet_gni_gini_gdp_hap_lit_sui_CO2_df, ip_merge_iso[["Internet_Price", "ISO", "Year"]], on=['ISO', 'Year'], how='outer')
merged['Internet Price Ratio'] = 100* merged['Internet_Price'] / merged['GNI per capita']
merged

,Country,Region,ISO,Year,"Individuals using the Internet, total (%)",GNI per capita,GINI Index,GDP Per Capita,Happiness Index,Literacy,Suicide Rates,CO2 Emissions,Internet_Price,Internet Price Ratio
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,520.0,NaN,554.594735,NaN,NaN,4.3,0.304230,NaN,NaN
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,550.0,NaN,621.912414,NaN,31.448851,4.1,0.408965,NaN,NaN
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,650.0,NaN,663.141053,NaN,NaN,4.0,0.335061,NaN,NaN
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,680.0,NaN,651.987862,4.04,NaN,4.0,0.298088,NaN,NaN
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,650.0,NaN,628.146804,NaN,NaN,3.9,0.283692,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2678,NaN,NaN,TWN,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.015246,NaN
2679,NaN,NaN,TWN,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.924705,NaN
2680,NaN,NaN,TWN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.594986,NaN
2681,NaN,NaN,TWN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.669785,NaN


Again there are a few rows with countries that doesn't appear in the World Bank datasets so we will drop those.

In [93]:
internet_gni_gini_gdp_hap_lit_sui_CO2_price_df = merged.dropna(subset=['Country'])

In [94]:
internet_gni_gini_gdp_hap_lit_sui_CO2_price_df.shape

(2672, 14)

At the end of this section the dataset is called internet_gni_gini_gdp_hap_lit_sui_CO2_price_df

## Voter Turnout <a name = "VoterTurnout"></a>

In [95]:
voter_turnout_df = pd.read_excel("data/idea_export_voter_turnout_database_world.xlsx")
voter_turnout_df.head()

,Country,ISO2,ISO3,year,Voter Turnout
0,Afghanistan,AF,AFG,2018-01-01,45.23%
1,Afghanistan,AF,AFG,2010-01-01,35.14%
2,Afghanistan,AF,AFG,2005-01-01,49.37%
3,Afghanistan,AF,AFG,2005-01-01,49.37%
4,Albania,AL,ALB,2021-01-01,46.32%


### Make sure the data in the columns are mergeable <a name ="renameAndReformatData"></a>

In [96]:
#I need to change the year column to include only the first four characters to show the year.
voter_turnout_df['year'] = voter_turnout_df['year'].str[:4]
voter_turnout_df['year']

0       2018
1       2010
2       2005
3       2005
4       2021
        ... 
2324    2000
2325    1995
2326    1979
2327    2013
2328    2023
Name: year, Length: 2329, dtype: object

In [97]:
#The output type is object, therefore it will need to be changed to Int64 in order to be merged with the main data set.
voter_turnout_df['year'] = voter_turnout_df['year'].astype('int64')

voter_turnout_df['year'].dtypes

dtype('int64')

In [98]:
# I need to remove all years before 2010 as these are not considered in the main data set
voter_turnout_df = voter_turnout_df[voter_turnout_df['year'] >= 2010]

#Check that the function has worked correctly.
voter_turnout_df

,Country,ISO2,ISO3,year,Voter Turnout
0,Afghanistan,AF,AFG,2018,45.23%
1,Afghanistan,AF,AFG,2010,35.14%
4,Albania,AL,ALB,2021,46.32%
5,Albania,AL,ALB,2017,46.76%
6,Albania,AL,ALB,2013,53.31%
...,...,...,...,...,...
2314,Zambia,ZM,ZMB,2016,56.03%
2315,Zambia,ZM,ZMB,2011,53.65%
2321,Zimbabwe,ZW,ZWE,2018,83.10%
2327,Zimbabwe,ZW,ZWE,2013,57.78%


### Remove NaN data <a name="RemoveNaNs"></a>

In [99]:
#Display all Na and Null data values
nan_count_per_column = voter_turnout_df.isna().sum()

# Print the count of NaN values in each column
print(nan_count_per_column)

Country          0
ISO2             2
ISO3             5
year             0
Voter Turnout    0
dtype: int64


In [100]:
# Display what the NaNs are for ISO3 because we will want to merge on them later.

nan_rows = voter_turnout_df[voter_turnout_df.isna().any(axis=1)]

# Print the rows with NaN values
nan_rows

,Country,ISO2,ISO3,year,Voter Turnout
1227,Kosovo,KV,NaN,2021,48.84%
1228,Kosovo,KV,NaN,2019,43.20%
1229,Kosovo,KV,NaN,2017,41.30%
1230,Kosovo,KV,NaN,2014,42.63%
1231,Kosovo,KV,NaN,2010,45.62%
1513,Namibia,NaN,NAM,2019,60.38%
1514,Namibia,NaN,NAM,2014,72.00%


### Remove Unecessary columns from the data <a name ="removecolumns"></a>

In [101]:
#Drop the ISO2 column now, as it is the only column with NaNs and it is not needed for the analysis as we will use ISO3 for the merge.
voter_turnout_df.drop(columns=['ISO2'], inplace=True)
voter_turnout_df

,Country,ISO3,year,Voter Turnout
0,Afghanistan,AFG,2018,45.23%
1,Afghanistan,AFG,2010,35.14%
4,Albania,ALB,2021,46.32%
5,Albania,ALB,2017,46.76%
6,Albania,ALB,2013,53.31%
...,...,...,...,...
2314,Zambia,ZMB,2016,56.03%
2315,Zambia,ZMB,2011,53.65%
2321,Zimbabwe,ZWE,2018,83.10%
2327,Zimbabwe,ZWE,2013,57.78%


In [102]:
#I need to rename ISO3 to ISO in order to complete the merge.
voter_turnout_df = voter_turnout_df.rename(columns={'ISO3': 'ISO'})

voter_turnout_df

,Country,ISO,year,Voter Turnout
0,Afghanistan,AFG,2018,45.23%
1,Afghanistan,AFG,2010,35.14%
4,Albania,ALB,2021,46.32%
5,Albania,ALB,2017,46.76%
6,Albania,ALB,2013,53.31%
...,...,...,...,...
2314,Zambia,ZMB,2016,56.03%
2315,Zambia,ZMB,2011,53.65%
2321,Zimbabwe,ZWE,2018,83.10%
2327,Zimbabwe,ZWE,2013,57.78%


In [103]:
#I need to rename year to Year in order to complete the merge.

voter_turnout_df = voter_turnout_df.rename(columns={'year': 'Year'})
voter_turnout_df

,Country,ISO,Year,Voter Turnout
0,Afghanistan,AFG,2018,45.23%
1,Afghanistan,AFG,2010,35.14%
4,Albania,ALB,2021,46.32%
5,Albania,ALB,2017,46.76%
6,Albania,ALB,2013,53.31%
...,...,...,...,...
2314,Zambia,ZMB,2016,56.03%
2315,Zambia,ZMB,2011,53.65%
2321,Zimbabwe,ZWE,2018,83.10%
2327,Zimbabwe,ZWE,2013,57.78%


In [104]:
#Need Voter Turnout as a float as currently will not be able to merge.
voter_turnout_df['Voter Turnout'] = voter_turnout_df['Voter Turnout'].str.rstrip('%')

voter_turnout_df['Voter Turnout'] = pd.to_numeric(voter_turnout_df['Voter Turnout'], errors='coerce')
voter_turnout_df

,Country,ISO,Year,Voter Turnout
0,Afghanistan,AFG,2018,45.23
1,Afghanistan,AFG,2010,35.14
4,Albania,ALB,2021,46.32
5,Albania,ALB,2017,46.76
6,Albania,ALB,2013,53.31
...,...,...,...,...
2314,Zambia,ZMB,2016,56.03
2315,Zambia,ZMB,2011,53.65
2321,Zimbabwe,ZWE,2018,83.10
2327,Zimbabwe,ZWE,2013,57.78


### Merge this data set with the main <a name="mergewithmain"></a>

In [105]:
#Merge the dataSets
merged_df = pd.merge(internet_gni_gini_gdp_hap_lit_sui_CO2_price_df,voter_turnout_df[["Voter Turnout", "Country","Year", "ISO"]], left_on = ['ISO', 'Year'],right_on =['ISO', 'Year'], how ='outer')
merged_df

,Country_x,Region,ISO,Year,"Individuals using the Internet, total (%)",GNI per capita,GINI Index,GDP Per Capita,Happiness Index,Literacy,Suicide Rates,CO2 Emissions,Internet_Price,Internet Price Ratio,Voter Turnout,Country_y
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,520.0,NaN,554.594735,NaN,NaN,4.3,0.304230,NaN,NaN,35.14,Afghanistan
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,550.0,NaN,621.912414,NaN,31.448851,4.1,0.408965,NaN,NaN,NaN,NaN
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,650.0,NaN,663.141053,NaN,NaN,4.0,0.335061,NaN,NaN,NaN,NaN
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,680.0,NaN,651.987862,4.04,NaN,4.0,0.298088,NaN,NaN,NaN,NaN
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,650.0,NaN,628.146804,NaN,NaN,3.9,0.283692,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2723,NaN,NaN,TKM,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.12,Turkmenistan
2724,NaN,NaN,VGB,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.50,"Virgin Islands, British"
2725,NaN,NaN,VGB,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.64,"Virgin Islands, British"
2726,NaN,NaN,VGB,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.58,"Virgin Islands, British"


In [106]:
pd.set_option('display.max_rows', 10)
country_info_dict = dict(zip(internet_gni_gini_gdp_hap_lit_sui_CO2_price_df['ISO'], zip(internet_gni_gini_gdp_hap_lit_sui_CO2_price_df['Country'], internet_gni_gini_gdp_hap_lit_sui_CO2_price_df['Region'])))
country_info_dict

for index, row in merged_df.iterrows():
    iso = row["ISO"]
    if iso in country_info_dict:
        country_name, region = country_info_dict[iso]
        if pd.isnull(row["Country_x"]):
            merged_df.at[index, "Country_x"] = country_name
        if pd.isnull(row["Region"]):
            merged_df.at[index, "Region"] = region
        if pd.isnull(row["ISO"]):
            merged_df.at[index, "ISO"] = iso_code

merged_df

,Country_x,Region,ISO,Year,"Individuals using the Internet, total (%)",GNI per capita,GINI Index,GDP Per Capita,Happiness Index,Literacy,Suicide Rates,CO2 Emissions,Internet_Price,Internet Price Ratio,Voter Turnout,Country_y
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,520.0,NaN,554.594735,NaN,NaN,4.3,0.304230,NaN,NaN,35.14,Afghanistan
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,550.0,NaN,621.912414,NaN,31.448851,4.1,0.408965,NaN,NaN,NaN,NaN
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,650.0,NaN,663.141053,NaN,NaN,4.0,0.335061,NaN,NaN,NaN,NaN
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,680.0,NaN,651.987862,4.04,NaN,4.0,0.298088,NaN,NaN,NaN,NaN
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,650.0,NaN,628.146804,NaN,NaN,3.9,0.283692,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2723,Turkmenistan,CIS,TKM,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.12,Turkmenistan
2724,NaN,NaN,VGB,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.50,"Virgin Islands, British"
2725,NaN,NaN,VGB,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.64,"Virgin Islands, British"
2726,NaN,NaN,VGB,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.58,"Virgin Islands, British"


In [107]:
pd.set_option('display.max_rows', None)
filtered_df = merged_df[merged_df["Country_x"].isnull()]
unique_country_y_count = filtered_df['Country_y'].value_counts()
unique_country_y_count

Country_y
Kosovo                     5
Cook Islands               4
Gibraltar                  4
Virgin Islands, British    4
Anguilla                   3
Aruba                      3
Bermuda                    3
Cayman Islands             3
Greenland                  3
Taiwan                     3
Faroe Islands              2
Montserrat                 2
Palau                      2
Name: count, dtype: int64

All territories and countries which don't appear in the world bank dataset, dropping them for the time being.

In [108]:
merged_df.dropna(subset=['Country_x'], inplace=True)
merged_df.drop(columns=['Country_y'], inplace=True)
merged_df.rename(columns={'Country_x': 'Country'}, inplace=True)

In [109]:
#Create a copy from which you can create the CSV
internet_gni_gini_gdp_hap_lit_sui_CO2_price_voterdb_df = merged_df.copy()

In [110]:
internet_gni_gini_gdp_hap_lit_sui_CO2_price_voterdb_df.shape

(2687, 15)

## DataFrame and Export <a name="finalexport"></a>
Exporting dataframe to csv.

In [111]:
internet_gni_gini_gdp_hap_lit_sui_CO2_price_voterdb_df.to_csv('internet_and_development_13122023.csv', index=False)

In [112]:
internet_gni_gini_gdp_hap_lit_sui_CO2_price_voterdb_df.head()

,Country,Region,ISO,Year,"Individuals using the Internet, total (%)",GNI per capita,GINI Index,GDP Per Capita,Happiness Index,Literacy,Suicide Rates,CO2 Emissions,Internet_Price,Internet Price Ratio,Voter Turnout
0,Afghanistan,Asia & Pacific,AFG,2010,4.000000,520.0,NaN,554.594735,NaN,NaN,4.3,0.304230,NaN,NaN,35.14
1,Afghanistan,Asia & Pacific,AFG,2011,5.000000,550.0,NaN,621.912414,NaN,31.448851,4.1,0.408965,NaN,NaN,NaN
2,Afghanistan,Asia & Pacific,AFG,2012,5.454545,650.0,NaN,663.141053,NaN,NaN,4.0,0.335061,NaN,NaN,NaN
3,Afghanistan,Asia & Pacific,AFG,2013,5.900000,680.0,NaN,651.987862,4.04,NaN,4.0,0.298088,NaN,NaN,NaN
4,Afghanistan,Asia & Pacific,AFG,2014,7.000000,650.0,NaN,628.146804,NaN,NaN,3.9,0.283692,NaN,NaN,NaN
